alfie pymongo use

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from pymongo import MongoClient
import matplotlib.pyplot as plt
import pandas as pd
import ast
import datetime
import seaborn as sns
import json
import numpy as np

import matplotlib as mpl

from datetime import datetime
from dateutil.parser import parse

#need to translate from wgs to bng to do spatial clustering
import pyproj

from pandas.io.json import json_normalize

#now to try out the clustering with this new data set
#now import DBSCAN clustering package
from sklearn.cluster import DBSCAN

In [ ]:
# specifies number of rows to show
pd.set_option('display.max_rows', 300) 

# specifies default number format to 4 decimal places
pd.options.display.float_format = '{:40,.4f}'.format 

# specifies that graphs should use ggplot styling
plt.style.use('ggplot') 

%matplotlib inline

## Some setups

In [ ]:
pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
%matplotlib inline

In [ ]:
from astropy.stats import RipleysKEstimator

In [ ]:
# load .env variable
# base_dir = 'D:\Documentos\GitHub\CityBlender'
# dotenv_file = os.path.join(base_dir, '.env')

base_dir = 'C:/Users/Alfie/Documents/MSc Smart Cities/Term 2/Spatial Data Capture Storage and Analysis/Coursework/Analysis Test/'
dotenv_file = os.path.join(base_dir, '.env')


# Continue loading stuff

if os.path.isfile(dotenv_file):
    load_dotenv(dotenv_file, verbose=True)
    
db_uri = os.getenv('DB_URI')

db_client = MongoClient(str(db_uri))


# choose a database to connect to (.london)
db_london = db_client.london
status = db_client.london.command('serverStatus')


# create the connections and build the local datasets
db_london_events = db_london['events']
db_london_artist = db_london['artists']

allLondonEvents = db_london_events.find()

londonEventsdf = pd.DataFrame(list(allLondonEvents))

# Some processing

In [ ]:
# Put this in the date format
londonEventsdf['date'] = [(datetime.strptime(londonEventsdf['date'][x], '%Y-%m-%d')) for x in range(len(londonEventsdf['date']))]

# Convert the dates into day name
londonEventsdf['date_name'] = [(londonEventsdf['date'][x].strftime("%A")) for x in range(len(londonEventsdf['date']))]

# Lat & Long
londonEventsdf['lat'] = [(londonEventsdf['location'][x]['lat']) for x in range(len(londonEventsdf['location']))]
londonEventsdf['lng'] = [(londonEventsdf['location'][x]['lng']) for x in range(len(londonEventsdf['location']))]

# Dates in the dataset

In [ ]:
#get min and max date of this dataset
print(min(londonEventsdf['date']))
print(max((londonEventsdf['date'])))

In [ ]:
# Define some common projections using EPSG codes
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
osgb36=pyproj.Proj("+init=EPSG:27700") # UK Ordnance Survey, 1936 datum

In [ ]:
#create a new pd without NaNs
londonEventsdfNoNan = londonEventsdf[londonEventsdf['lat'].notnull()]

#make new columns for the new latlng
londonEventsdfNoNan['BNGnorthing'] = ""
londonEventsdfNoNan['BNGeasting'] = ""


londonEventsdfNoNan = londonEventsdfNoNan.reset_index()

In [ ]:
#now run through the dataset and transform those coordinates
origLat = []
origLng = []

for i in londonEventsdfNoNan['index']:
    # print(i)
    try:
        origLat.append(londonEventsdfNoNan['lat'][i])
        origLng.append(londonEventsdfNoNan['lng'][i])
        
    except:
        origLat.append(np.nan)
        origLng.append(np.nan)

In [ ]:
lat,lon = pyproj.transform(wgs84,osgb36,origLat,origLng)
    
londonEventsdfNoNan['BNGnorthing'] = lat
londonEventsdfNoNan['BNGeasting'] = lon

In [ ]:
londonEventsdfNoNan

In [ ]:
londonEventsdfNoNan['BNGnorthing'] = pd.to_numeric(londonEventsdfNoNan['BNGnorthing'])
londonEventsdfNoNan['BNGeasting'] = pd.to_numeric(londonEventsdfNoNan['BNGeasting'])

In [ ]:
minLatLdn = min(londonEventsdfNoNan['BNGnorthing'])
minLngLdn = min(londonEventsdfNoNan['BNGeasting'])
maxLatLdn = max(londonEventsdfNoNan['BNGnorthing'])
maxLngLdn = max(londonEventsdfNoNan['BNGeasting'])

In [ ]:
londonEventsdfNoNan = londonEventsdfNoNan[londonEventsdfNoNan['BNGnorthing'].notnull()]

In [ ]:
londonEventsdfNoNan = londonEventsdfNoNan.reset_index()

In [ ]:
# Make a function to subset the genere depending on which dataset
def subset_by_genere(db = londonEventsdfNoNan, lib = 'spotify', words = 'reggae'):
    return db[db[lib].astype(str).str.contains(words, case=False)]

In [ ]:
reggaeEventsdf = subset_by_genere()

reggaeEventsdf2 = subset_by_genere(lib = 'lastfm')
allReggaeEvents = pd.concat([reggaeEventsdf,reggaeEventsdf2])

In [ ]:
#get rid of duplicates
allReggaeEvents = allReggaeEvents.drop_duplicates(subset='_id', keep='last')

In [ ]:
print(minLatLdn)
print(minLngLdn)
print(maxLatLdn)
print(maxLngLdn)

In [ ]:
#now set up the ripley's k estimator object
Kest = RipleysKEstimator(area=2,x_max=maxLngLdn, y_max=maxLatLdn, x_min=minLngLdn, y_min=minLatLdn)

In [ ]:
#need to put the lat and lng as an array
locsAsArray = londonEventsdfNoNan[['BNGnorthing', 'BNGeasting']].values

In [ ]:
locsAsArray

In [ ]:
#now get reggae locs
regLocsAsArray = allReggaeEvents[['BNGnorthing', 'BNGeasting']].values

In [ ]:
regLocsAsArray

In [ ]:
#now to plot and have a look
r = np.linspace(0, 2500, 50)
plt.plot(r, Kest.poisson(r), color='green', ls=':', label=r'$K_{pois}$')
plt.plot(r, Kest(data=locsAsArray, radii=r, mode='none'), color='red', ls='--',
         label=r'$K_{un}$')
plt.plot(r, Kest(data=locsAsArray, radii=r, mode='translation'), color='black',
         label=r'$K_{trans}$')
plt.plot(r, Kest(data=locsAsArray, radii=r, mode='ohser'), color='blue', ls='-.',
         label=r'$K_{ohser}$')
plt.plot(r, Kest(data=locsAsArray, radii=r, mode='var-width'), color='green',
         label=r'$K_{var-width}$')
plt.plot(r, Kest(data=locsAsArray, radii=r, mode='ripley'), color='yellow',
         label=r'$K_{ripley}$')

In [ ]:
#try out a dbscan of all reggae
#create new dbscan cluster object
dbscan = DBSCAN(eps=500, min_samples=3)

In [ ]:
#get only locations from the reggae set
#get just locations
#use the locsArray again

dbscan1stRun = dbscan.fit(locsAsArray)

In [ ]:
#get the labels of the groups
dbscan_labels = dbscan.labels_

In [ ]:
dbscan_labels

In [ ]:
#now add the labels back into the reggae dataframe
#need to reindex allReggaeEvents for it to work
#londonEventsdfNoNan = londonEventsdfNoNan.reset_index()

In [ ]:
londonEventsdfNoNan['dbscanLabels1'] = pd.DataFrame(dbscan_labels)

In [ ]:
#now give it a plot
#define colorscheme

londonEventsdfNoNan.plot.scatter(y='lng', x='lat', c='dbscanLabels1', )

In [ ]:
#try clustering out on the whole dataset
dbscan2ndRun = dbscan.fit(locsAsArray)

In [ ]:
londonEventsdf